In [1]:
# -*- coding: utf-8 -*-
from requests.exceptions import RequestException
from bs4 import BeautifulSoup
import requests
import re
import time
from multiprocessing import Pool
import csv
import threading

In [ ]:

def get_one_page(url):
    try:
        header ={
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36'
        }
        response = requests.get(url,headers=header)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

    
def parse_one_page(html,offset):
    result = {}
    soup = BeautifulSoup(str(html),'lxml')
    
    name = soup.select('#pageData')
    name = re.findall('movieName":"(.*?)"',str(name),re.S)
    result['n'] = str(name).replace("['","").replace("']","")
    
    for sex in soup.select('.stackcolumn-bar')[:1]:
        result['m'] = round(float(sex.attrs['style'].replace("width: ","").replace('%',''))/100,3)        
    for sex in soup.select('.stackcolumn-bar')[1:2]:
        result['fm'] = round(float(sex.attrs['style'].replace("width: ","").replace('%',''))/100,3)

        
    for age in soup.select('.bar-group g')[:1]:
        result['un20'] = round(float(age.get_text().replace('%',''))/100,3)        
    for age in soup.select('.bar-group g')[1:2]:
        result['20to24'] = round(float(age.get_text().replace('%',''))/100,3)      
    for age in soup.select('.bar-group g')[2:3]:
        result['25to29'] = round(float(age.get_text().replace('%',''))/100,3)   
    for age in soup.select('.bar-group g')[3:4]:
        result['30to34'] = round(float(age.get_text().replace('%',''))/100,3)     
    for age in soup.select('.bar-group g')[4:5]:
        result['35to39'] = round(float(age.get_text().replace('%',''))/100,3)
    for age in soup.select('.bar-group g')[5:6]:
        result['ab40'] = round(float(age.get_text().replace('%',''))/100,3)
   

    for edu in soup.select('.stackcolumn-bar')[2:3]:
        result['dg_h'] = round(float(edu.attrs['style'].replace("width: ","").replace("%",""))/100,3)
    for edu in soup.select('.stackcolumn-bar')[3:]:
        result['dg_l'] = round(float(edu.attrs['style'].replace("width: ","").replace("%",""))/100,3)
   
    for area in soup.select('.linebars-value')[3:]:
        result['ct1'] = round(float(area.get_text().replace("%",""))/100,3)
    for area in soup.select('.linebars-value')[:1]:
        result['ct2'] = round(float(area.get_text().replace("%",""))/100,3)
    for area in soup.select('.linebars-value')[2:3]:
        result['ct3'] = round(float(area.get_text().replace("%",""))/100,3)
    for area in soup.select('.linebars-value')[1:2]:
        result['ct4'] = round(float(area.get_text().replace("%",""))/100,3)
        
    try:
        for occupation in soup.select('.pie-label-line text')[:3]: 
            result[occupation.get_text()[:2]] = round(float(occupation.get_text()[2:].replace("%",""))/100,2)

        result['oc_w'] = result.pop('白领')
        result['oc_o'] = result.pop('其他')
        result['oc_s'] = result.pop('学生')
    except KeyError:
        print(None)
        
    try:
        for preference in soup.select('.pie-label-line text')[3:]:
            result[preference.get_text()[:2]] = round(float(preference.get_text()[2:].replace("%",""))/100,2)

        result['pf_ac'] = result.pop("动作")
        result['pf_co'] = result.pop("喜剧")
        result['pf_ro'] = result.pop("爱情")
        result['pf_sc'] = result.pop("科幻")
        result['pf_an'] = result.pop("动画")
    except KeyError:
        print(None)
        
    print(result)
    return result



# def writein(result):
#     #打开文件，追加a
#     try:
#         out = open('yonghuhuaxiang3.csv','a', newline='')
#         #设定写入模式
#         csv_write = csv.writer(out,dialect = 'excel')
#         #写入具体内容
#         csv_write.writerow(result)
#         print ("write over")
#     except UnicodeError:
#         print("go on")

def main(offset):
    url = 'https://piaofang.maoyan.com/movie/'+str(offset)+'/wantindex?'
    html = get_one_page(url)
    result = parse_one_page(html,offset)


def readin():
    for line in open("2016id.txt"):
        main(line.replace("\n",""))

if __name__ == '__main__':
    for i in range(1000):
        t = threading.Thread(target=readin())
        thread_list.append(t)
    for t in thread_list:
        t.start()

{'n': '大鱼海棠', 'm': 0.394, 'fm': 0.606, 'un20': 0.118, '20to24': 0.338, '25to29': 0.239, '30to34': 0.128, '35to39': 0.087, 'ab40': 0.089, 'dg_h': 0.304, 'dg_l': 0.696, 'ct1': 0.126, 'ct2': 0.426, 'ct3': 0.193, 'ct4': 0.256, 'oc_w': 0.3, 'oc_o': 0.54, 'oc_s': 0.16, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0.14, 'pf_an': 0.05}
{'n': '你的名字。', 'm': 0.483, 'fm': 0.517, 'un20': 0.222, '20to24': 0.422, '25to29': 0.178, '30to34': 0.055, '35to39': 0.035, 'ab40': 0.088, 'dg_h': 0.343, 'dg_l': 0.657, 'ct1': 0.143, 'ct2': 0.439, 'ct3': 0.185, 'ct4': 0.233, 'oc_w': 0.22, 'oc_o': 0.49, 'oc_s': 0.28, 'pf_ac': 0.32, 'pf_co': 0.13, 'pf_ro': 0.28, 'pf_sc': 0.15, 'pf_an': 0.05}
{'n': '逗鸟外传：萌宝满天飞', 'm': 0.377, 'fm': 0.623, 'un20': 0.065, '20to24': 0.239, '25to29': 0.255, '30to34': 0.227, '35to39': 0.137, 'ab40': 0.077, 'dg_h': 0.285, 'dg_l': 0.715, 'ct1': 0.129, 'ct2': 0.426, 'ct3': 0.197, 'ct4': 0.248, 'oc_w': 0.34, 'oc_o': 0.55, 'oc_s': 0.11, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_

{'n': '功夫熊猫3', 'm': 0.463, 'fm': 0.537, 'un20': 0.071, '20to24': 0.284, '25to29': 0.265, '30to34': 0.171, '35to39': 0.115, 'ab40': 0.094, 'dg_h': 0.303, 'dg_l': 0.697, 'ct1': 0.139, 'ct2': 0.438, 'ct3': 0.193, 'ct4': 0.229, 'oc_w': 0.37, 'oc_o': 0.53, 'oc_s': 0.11, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0.15, 'pf_an': 0.06}
{'n': '熊出没之熊心归来', 'm': 0.39, 'fm': 0.61, 'un20': 0.045, '20to24': 0.192, '25to29': 0.254, '30to34': 0.263, '35to39': 0.163, 'ab40': 0.083, 'dg_h': 0.243, 'dg_l': 0.757, 'ct1': 0.109, 'ct2': 0.416, 'ct3': 0.219, 'ct4': 0.256, 'oc_w': 0.37, 'oc_o': 0.57, 'oc_s': 0.07, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0.14, 'pf_an': 0.08}
{'n': '神战：权力之眼', 'm': 0.54, 'fm': 0.46, 'un20': 0.058, '20to24': 0.305, '25to29': 0.309, '30to34': 0.155, '35to39': 0.09, 'ab40': 0.083, 'dg_h': 0.295, 'dg_l': 0.705, 'ct1': 0.15, 'ct2': 0.453, 'ct3': 0.19, 'ct4': 0.207, 'oc_w': 0.4, 'oc_o': 0.51, 'oc_s': 0.09, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0

{'n': '龙珠Z：复活的弗利萨', 'm': 0.632, 'fm': 0.368, 'un20': 0.088, '20to24': 0.322, '25to29': 0.27, '30to34': 0.164, '35to39': 0.094, 'ab40': 0.061, 'dg_h': 0.267, 'dg_l': 0.733, 'ct1': 0.129, 'ct2': 0.429, 'ct3': 0.215, 'ct4': 0.226, 'oc_w': 0.33, 'oc_o': 0.54, 'oc_s': 0.13, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0.16, 'pf_an': 0.06}
{'n': '封神传奇', 'm': 0.489, 'fm': 0.511, 'un20': 0.078, '20to24': 0.297, '25to29': 0.284, '30to34': 0.16, '35to39': 0.101, 'ab40': 0.08, 'dg_h': 0.235, 'dg_l': 0.765, 'ct1': 0.134, 'ct2': 0.415, 'ct3': 0.199, 'ct4': 0.251, 'oc_w': 0.32, 'oc_o': 0.59, 'oc_s': 0.1, 'pf_ac': 0.32, 'pf_co': 0.12, 'pf_ro': 0.28, 'pf_sc': 0.15, 'pf_an': 0.05}
{'n': '三人行', 'm': 0.44, 'fm': 0.56, 'un20': 0.085, '20to24': 0.337, '25to29': 0.301, '30to34': 0.133, '35to39': 0.07, 'ab40': 0.074, 'dg_h': 0.278, 'dg_l': 0.722, 'ct1': 0.145, 'ct2': 0.429, 'ct3': 0.196, 'ct4': 0.23, 'oc_w': 0.35, 'oc_o': 0.54, 'oc_s': 0.12, 'pf_ac': 0.32, 'pf_co': 0.14, 'pf_ro': 0.29, 'pf_sc': 0.15